<a href="https://colab.research.google.com/github/alexlinapp/proofLLM/blob/main/attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from importlib.metadata import version

print("torch version:", version("torch"))

torch version: 2.6.0+cu124


In [122]:
import torch
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)
query = inputs[1]
attention_scores = torch.empty(inputs.shape[0])
for i, xi in enumerate(inputs):
  attention_scores[i] = torch.dot(query, xi)
attention_scores, inputs.shape
attention_scores_normalized = attention_scores / torch.sum(attention_scores)
attention_scores_normalized, attention_scores, query.shape

(tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656]),
 tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865]),
 torch.Size([3]))

In [13]:
context_vec_2 = torch.zeros(query.shape)
for i, xi in enumerate(inputs):
  context_vec_2 += attention_scores_normalized[i] * xi
  print(attention_scores_normalized[i].shape, xi.shape) # attention_scores_normalized is a vector containing the attention weights of each item imbedding
  # , xi is the actual input/item embedding
context_vec_2.shape


torch.Size([]) torch.Size([3])
torch.Size([]) torch.Size([3])
torch.Size([]) torch.Size([3])
torch.Size([]) torch.Size([3])
torch.Size([]) torch.Size([3])
torch.Size([]) torch.Size([3])


torch.Size([3])

In [34]:
# compute all context vectors
atten_scores = torch.empty(inputs.shape[0], inputs.shape[0])
for i, x_i in enumerate(inputs):
  for j, x_j in enumerate(inputs):
    atten_scores[i, j] = torch.dot(x_i, x_j)
print(atten_scores)
# using for loops is too slow, notice we can use matrix multiplication
atten_score_matrix = inputs @ inputs.T
atten_score_matrix == atten_scores, atten_score_matrix
atten_weights = torch.softmax(atten_scores, dim=-1)
allcontext_vecs = atten_weights @ inputs
# To see this,
# --------------------
#   w1 = {w_{1,1}, w_{1,2}, ..., w_{1,n}} |   | a_1 = {a_{1,1}, a{1,2}, a_{1,3}}
#   w2 = .....                            |   | a_2 = {a{2,1}, ...}
#                                             |
allcontext_vecs

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

In [141]:
# Self-attention with trainable weights
x_2 = inputs[1];  # second input element
# print(x_2.shape)
print("x2: ", x_2)
d_in = inputs.shape[1]  # input embedding dimension
d_out = 2               # output embedding dimension
torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out))
W_key = torch.nn.Parameter(torch.rand(d_in, d_out))
W_value = torch.nn.Parameter(torch.rand(d_in, d_out))
print(W_query, W_value, W_key)
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value
# print(query_2.shape, key_2.shape, value_2.shape)
print("qyery2: ", query_2)
keys = inputs @ W_key
values = inputs @ W_value
# print(keys.shape, values.shape)
keys2 = keys[1]
atten_score_22 = query_2.dot(keys2)#torch.dot(query_2, keys2)
# print("atten_score_22", atten_score_22)
atten_score_2 = query_2 @ keys.T
# print("query_2 shape: ", query_2.shape)
# print("keys shape: ", keys.shape)
print("atten_score_2: ", atten_score_2)
d_k = keys.shape[1]
atten_weights_2 = torch.softmax(atten_score_2 /  d_k**0.5, dim = -1) #  sqrt used to prevent sharp gradients during training
print(atten_weights_2)
context_vec_2 = atten_weights_2 @ values
print(context_vec_2)
# print(keys, keys2)

x2:  tensor([0.5500, 0.8700, 0.6600])
Parameter containing:
tensor([[0.2961, 0.5166],
        [0.2517, 0.6886],
        [0.0740, 0.8665]], requires_grad=True) Parameter containing:
tensor([[0.0756, 0.1966],
        [0.3164, 0.4017],
        [0.1186, 0.8274]], requires_grad=True) Parameter containing:
tensor([[0.1366, 0.1025],
        [0.1841, 0.7264],
        [0.3153, 0.6871]], requires_grad=True)
qyery2:  tensor([0.4306, 1.4551], grad_fn=<SqueezeBackward4>)
atten_score_2:  tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440],
       grad_fn=<SqueezeBackward4>)
tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820],
       grad_fn=<SoftmaxBackward0>)
tensor([0.3061, 0.8210], grad_fn=<SqueezeBackward4>)
